# smallest-free-number
A coding challenge to find the smallest non-negative integer that is not part of a given set of non-negative integers.

Thanks to [Nicolas Rinaudo](https://github.com/abcoates/smallest-free-number.git) for suggesting this problem for a coding challenge.

## Description
This is a simplification a general problem, find the 'least XXX' object that is not already used, where 'XXX' is some arbitrary measurement dimension such as 'large', 'expensive', etc.

In this minimal version, you are given a set (i.e. an **unordered** set) of non-negative integers, and you have to find the smallest non-negative integer that is **not** a part of the set.

## Examples
 * \[0, 1, 2, 3, 5\] => 4
 * \[2, 1, 0\] => 3
 * \[20, 10, 30\] => 0

## Special Note
You **may** use AI to help you write the code.  As AI coding companions are inevitable, we might as well all start practicising how to use them.  **However**, if you have used AI to help you write your code, please make that clear in your solution.

## Stretch Goal
Finding a solution is straightforward, but can you find a solution that only takes linear time?  Measure how the time taken for your solution varies as the size of the set is increased, and see how close you can get to the time taken growing linearly with the size of the set.  Create a graph of the time measurements again set size.

You will need to generate unordered sets of numbers that are sufficiently large to make the solution time sufficiently measurable.

Your results may vary with the **density** of the sets, i.e. with the percentage of unused numbers in the range from zero to the largest number in the set.

## Solution - F#

### Solution #1 - Brute Force

As a first solution, let's try something "brute force" - sort the set, then walk up the integers from zero until we find one that isn't in the set.

In [37]:
let solution1 (intset: int Set): int =
    let sortedset = intset |> Set.toList |> List.sort
    let rec find (sortedlist: int list) (nextValue: int) =
        match sortedlist with
        | [] -> nextValue
        | other ->
            let listhead = sortedlist |> List.head
            if (nextValue < listhead)
            then nextValue
            elif (nextValue = listhead)
            then find (sortedlist |> List.tail) (nextValue+1)
            else find(sortedlist |> List.tail) nextValue
    find sortedset 0

In [38]:
solution1 ([0;1;2;3;5] |> Set.ofList)

4

In [39]:
solution1 ([0;1;2] |> Set.ofList)

3

In [40]:
solution1 ([10;20;30] |> Set.ofList)

0

So 'solution1' works correctly.  However, how performant is it?

We'll need a graphing package - 'Plotly' will do nicely.

In [41]:
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

open XPlot.Plotly

Installed Packages XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

In [58]:
#r "nuget: MathNet.Numerics"

open MathNet.Numerics

Installed Packages MathNet.Numerics, 5.0.0

We'll need functions that can generate random sets of non-negative integers.

In [42]:
open System

let randomGenerator = Random()

// Generate a random integer between 1 and 100 (inclusive)
// let randomInt = random.Next(1, 101)

In [43]:
let generateSet (maximum:int) (length:int) =
    
    let rec iterate (currentSet: int Set) =
        match currentSet with
        | finished when (finished |> Set.count) >= length -> finished
        | unfinished ->
            let newVal = randomGenerator.Next(0, maximum+1)
            iterate (currentSet |> Set.add newVal)
    iterate ([] |> Set.ofList)

In [44]:
generateSet 10 5

Count,5
(values),"[ 2, 4, 5, 9, 10 ]"


In [45]:
solution1 (generateSet 10 5)

0

In [46]:
generateSet 5 5

Count,5
(values),"[ 1, 2, 3, 4, 5 ]"


In [47]:
solution1 (generateSet 5 5)

4

In [48]:
generateSet 5 6

Count,6
(values),"[ 0, 1, 2, 3, 4, 5 ]"


In [49]:
solution1 (generateSet 5 6)

6

Note that F# sorts sets, but these are sets, not sorted lists.  My solutions won't make any use of F# sets being naturally sorted.

The following function can be used to time how long a solution takes to run.  It averages the time over multiple runs.

In [50]:
let timeSolution (maximum: int) (length: int) (solution: int Set -> int): int*int*int =
    let timeOnce (): int*int*int*int =
        let sw = System.Diagnostics.Stopwatch.StartNew()
        let result = solution (generateSet maximum length)
        sw.Stop()
        (maximum, length, result, sw.Elapsed.Microseconds)
    let timings = {0..(maximum*length)} |> Seq.map (fun iter -> // do enough iterations to get a stable result
        let (_, _, _, timing) = timeOnce ()
        timing
    )
    (maximum, length, timings |> Seq.map float |> Seq.average |> round |> int)

In [51]:
timeSolution 100 50 solution1

Item1,100
Item2,50
Item3,21


Let's create timings (in microseconds) for set sizes from 50 to 300.

In [52]:
let lengths = {50..50..300} |> Seq.toList
let maximums = lengths |> List.map (fun n -> 2*n)
let pairs: (int*int) list = List.zip maximums lengths
let timings = pairs |> List.map (fun pair ->
    let (maximum, length) = pair
    let (_, _, timing) = timeSolution maximum length solution1
    timing
)

In [53]:
List.zip lengths timings

[ (50, 24), (100, 42), (150, 75), (200, 113), (250, 158), (300, 216) ] HeadOrDefault (50, 24) Item1 50 Item2 24 TailOrNull [ (100, 42), (150, 75), (200, 113), (250, 158), (300, 216) ] HeadOrDefault (100, 42) Item1 100 Item2 42 TailOrNull [ (150, 75), (200, 113), (250, 158), (300, 216) ] HeadOrDefault (150, 75) Item1 150 Item2 75 TailOrNull [ (200, 113), (250, 158), (300, 216) ] HeadOrDefault (200, 113) Item1 200 Item2 113 TailOrNull [ (250, 158), (300, 216) ] HeadOrDefault (250, 158) Item1 250 Item2 158 TailOrNull [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) Head (250, 158) Item1 250 Item2 158 Tail [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) (values) index value 0 (250, 158) Item1 250 Item2 158 1 (300, 216) Item1 300 Item2 216 Head (200, 113) Item1 200 Item2 113 Tail [ (250, 158), (300, 216) ] HeadOrDefault (250, 158) Item1 250 Item2 158 TailOrNull [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) Head (250, 158) Item1 250 Item2 158 Tail [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) (values) index value 0 (250, 158) Item1 250 Item2 158 1 (300, 216) Item1 300 Item2 216 (values) index value 0 (200, 113) Item1 200 Item2 113 1 (250, 158) Item1 250 Item2 158 2 (300, 216) Item1 300 Item2 216 Head (150, 75) Item1 150 Item2 75 Tail [ (200, 113), (250, 158), (300, 216) ] HeadOrDefault (200, 113) Item1 200 Item2 113 TailOrNull [ (250, 158), (300, 216) ] HeadOrDefault (250, 158) Item1 250 Item2 158 TailOrNull [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) Head (250, 158) Item1 250 Item2 158 Tail [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) (values) index value 0 (250, 158) Item1 250 Item2 158 1 (300, 216) Item1 300 Item2 216 Head (200, 113) Item1 200 Item2 113 Tail [ (250, 158), (300, 216) ] HeadOrDefault (250, 158) Item1 250 Item2 158 TailOrNull [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) Head (250, 158) Item1 250 Item2 158 Tail [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) (values) index value 0 (250, 158) Item1 250 Item2 158 1 (300, 216) Item1 300 Item2 216 (values) index value 0 (200, 113) Item1 200 Item2 113 1 (250, 158) Item1 250 Item2 158 2 (300, 216) Item1 300 Item2 216 (values) index value 0 (150, 75) Item1 150 Item2 75 1 (200, 113) Item1 200 Item2 113 2 (250, 158) Item1 250 Item2 158 3 (300, 216) Item1 300 Item2 216 Head (100, 42) Item1 100 Item2 42 Tail [ (150, 75), (200, 113), (250, 158), (300, 216) ] HeadOrDefault (150, 75) Item1 150 Item2 75 TailOrNull [ (200, 113), (250, 158), (300, 216) ] HeadOrDefault (200, 113) Item1 200 Item2 113 TailOrNull [ (250, 158), (300, 216) ] HeadOrDefault (250, 158) Item1 250 Item2 158 TailOrNull [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) Head (250, 158) Item1 250 Item2 158 Tail [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) (values) index value 0 (250, 158) Item1 250 Item2 158 1 (300, 216) Item1 300 Item2 216 Head (200, 113) Item1 200 Item2 113 Tail [ (250, 158), (300, 216) ] HeadOrDefault (250, 158) Item1 250 Item2 158 TailOrNull [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) Head (250, 158) Item1 250 Item2 158 Tail [ (300, 216) ] HeadOrDefault (300, 216) TailOrNull [ ] Head (300, 216) Tail [ ] (values) index value 0 (300, 216) (values) index value 0 (250, 158) Item1 250 Item2 158 1 (300, 216) Item1 300 Item2 216 (values) index value 0 (200, 113) Item1 200

Now let's try fitting an O(n) curve to the results.  I found I had to set 'p0init', 'p1init' and 'p2init' manually to give 'Fit.Curve' any chance to find a close fit.

In [134]:
let lengthsFloat = lengths |> List.map float |> Array.ofList
let timingsFloat = timings |> List.map float |> Array.ofList

let fitFunc (p0:float) (p1:float) (p2:float) (x:float) =
    // printfn "p0 = %f, p1 = %f, p2 = %f, x = %f" p0 p1 p2 x
    p0 + p1 * x**p2

let first<'a> (arr: 'a[]) = arr[0]
let last<'a> (arr: 'a[]) = arr[arr.Length - 1]

let p0init = 10.
let p1init = 0.01
let p2init = 1.65
let tolerance = 1.
let maxIterations = 1000
(p0init, p1init, p2init)

Item1,10
Item2,0.01
Item3,1.65


In [135]:
let (p0, p1, p2) = Fit.Curve(lengthsFloat, timingsFloat, fitFunc, p0init, p1init, p2init, tolerance, maxIterations).ToTuple()
(p0, p1, p2)

Item1,10.019675925925924
Item2,0.010283564814814811
Item3,1.731163194444444


OK, let's plot the measurements and the fit together.

In [136]:
let measurementCurve = Scatter(x = lengths, y = timings, name="Measurements")

let fitLengths = {(lengths |> List.min)..(lengths |> List.max)} |> Seq.map float |> Array.ofSeq
let fitTimings = fitLengths |> Array.map (fun x -> fitFunc p0 p1 p2 x)
let fitCurve = Scatter(x = fitLengths, y = fitTimings, name="O(n) Fit Curve")

[measurementCurve; fitCurve]
|> Chart.Plot
|> Chart.WithTitle("Solution Time vs. Set Size, Set Density = 50%")
|> Chart.WithXTitle("Set Size")
|> Chart.WithYTitle("Solution Time in Microsec")

<!DOCTYPE html>

So, approximately, the order of 'solution1' is 1.7.

## Appendix

**Note from Nicolas** If your set length is N, then your solution value must be in the range 0..N.